![](../../../../Documents/Screenshots/screenshot.286.jpg)
![](../../../../Documents/Screenshots/screenshot.284.jpg)


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

NEURAL_NETWORK_TYPE = 'convolutional'
shape = (96, 96, 1)


In [3]:
from getNeuralNetworkModel import get_neural_network_model

model = get_neural_network_model(NEURAL_NETWORK_TYPE)
model.summary()

In [4]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 102, 102, 1)      0         
 2D)                                                             
                                                                 
 conv1 (Conv2D)              (None, 100, 100, 64)      640       
                                                                 
 bn_conv1 (BatchNormalizatio  (None, 100, 100, 64)     256       
 n)                                                              
                                                                 
 activation (Activation)     (None, 100, 100, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 33, 33, 64)       0         
 )                                                               
                                                        

In [5]:
data = pd.read_feather('datasets/second_augmented_data.feather').copy()
data

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,64.678350,34.973984,31.766794,33.798576,59.095234,35.561687,69.674300,36.443310,36.468163,34.386280,...,53.781040,55.568880,70.237150,35.292492,69.355400,46.165478,64.947290,45.871563,76.114190,0.2705 0.298 0.3176 0.3843 0.447 0.4314 0.4314...
1,65.505260,36.810040,29.923138,38.468390,58.871530,37.413020,74.702430,37.111530,35.803295,38.317802,...,55.053490,71.083920,69.226036,28.113882,71.940080,48.468080,67.115295,49.975840,84.604866,1.0 1.0 1.0 1.0 0.992 1.0 0.7334 0.2235 0.1726...
2,65.537030,37.160828,30.592258,36.814840,60.001550,37.852800,71.073290,37.852800,35.781677,37.852800,...,57.574450,61.731100,77.642330,33.706066,76.258064,46.577805,72.213680,46.567740,84.692130,1.5044792 1.5276556 1.5124711 1.5124711 1.5124...
3,67.524620,39.423010,27.345154,40.703670,60.000988,39.903340,75.848580,40.863670,37.110104,40.383340,...,58.310596,63.061770,78.145320,33.063587,78.801155,48.308453,74.866806,48.964287,80.440414,0.1098 0.0902 0.0863 0.0902 0.0863 0.0863 0.07...
4,69.368210,35.821877,27.131690,36.711098,61.587730,36.711098,77.592870,36.266460,34.689823,37.600266,...,55.828724,68.701180,82.281950,29.799295,84.060190,47.360830,72.500890,48.472042,94.952450,0.8901275 0.87121457 0.88536876 0.9761508 0.93...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12835,70.335531,45.014747,28.937791,37.715209,63.305459,42.455028,77.163528,46.218708,37.071057,36.508967,...,59.177164,61.603550,79.559263,26.959501,73.890724,44.155041,71.201909,41.872748,86.641613,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.3882762 0.31...
12836,68.865864,53.509493,44.890221,25.496495,64.246201,49.403304,74.321977,60.391891,49.226569,32.396360,...,54.640618,42.518316,76.304622,18.739519,48.613704,31.545283,62.755458,22.038553,70.732551,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
12837,67.084800,37.864640,32.048000,39.294720,60.649600,37.864640,73.163200,38.222160,38.125600,39.652240,...,56.813520,67.084800,74.689840,35.623200,75.404880,48.493600,71.114560,48.851200,80.410400,36.74599999189377 16.74599999189377 3.24599999...
12838,68.926765,40.364292,28.215250,37.268463,60.543373,41.268383,79.173140,42.145077,36.489060,39.898550,...,60.520744,67.022240,78.620285,25.992792,75.544210,43.974277,76.403700,43.995537,83.684060,0.714 0.6665 0.6626 0.6353 0.6587 0.6704 0.647...


In [6]:
from sklearn.model_selection import train_test_split

data_in_batches = np.array_split(data, 5)

index = 0
for batch in data_in_batches:
    images = batch['Image'].str.split(' ', expand=True).astype(np.float32).to_numpy().reshape(-1, 96, 96, 1)
    points = batch.iloc[:, :-1].to_numpy()

    train_x, test_x, train_y, test_y = train_test_split(images, points, train_size=0.8)

    model.fit(train_x, train_y, epochs=500, validation_data=(test_x, test_y))
    model.save(f'models/facialPointsPredictor_{NEURAL_NETWORK_TYPE}.h5')

    index += 1

    print(f'-----------------------------------------------------------------------')
    print(f'Batch {index} completed')
    print(f'-----------------------------------------------------------------------')


Epoch 1/500
65/65 [==============================] - 20s 57ms/step - loss: 331.8722 - accuracy: 0.3627 - val_loss: 1448.5557 - val_accuracy: 0.6187
Epoch 2/500
65/65 [==============================] - 3s 43ms/step - loss: 148.0042 - accuracy: 0.4698 - val_loss: 727.2751 - val_accuracy: 0.6187
Epoch 3/500
65/65 [==============================] - 3s 42ms/step - loss: 144.9501 - accuracy: 0.5029 - val_loss: 289.3633 - val_accuracy: 0.6187
Epoch 4/500
65/65 [==============================] - 3s 43ms/step - loss: 146.6987 - accuracy: 0.5180 - val_loss: 259.2032 - val_accuracy: 0.6187
Epoch 5/500
65/65 [==============================] - 3s 42ms/step - loss: 143.2709 - accuracy: 0.5414 - val_loss: 344.1561 - val_accuracy: 0.6187
Epoch 6/500
65/65 [==============================] - 3s 43ms/step - loss: 144.5877 - accuracy: 0.5560 - val_loss: 147.5779 - val_accuracy: 0.6187
Epoch 7/500
65/65 [==============================] - 3s 43ms/step - loss: 143.5727 - accuracy: 0.5652 - val_loss: 125.7769


KeyboardInterrupt



In [4]:
from tensorflow.keras.models import load_model

model = load_model('models/facialPointsPredictor_convolutional.h5')


def get_points(image):
    image = tf.reshape(image, (96, 96, 1))
    points = model.predict(np.array([image]))[0]

    return points[0::2]*255, points[1::2]*255

In [5]:
import cv2

cap = cv2.VideoCapture(0)
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
glassesCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_lowerbody.xml')

if not cap.isOpened():
    print("Cannot open camera")
    exit()

cont = 0
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here

    frame = cv2.flip(frame, 90)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        frame_cara = cv2.resize(frame[y:y + h, x:x + w], (96, 96)) / 255

        points_x, points_y = get_points(frame_cara)

        points = zip(points_x, points_y)

        # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

        for point in points:
            #resize points coordinates to original dataframe
            point = (point[0] * w / 96, point[1] * h / 96)

            #draw cirlce
            cv2.circle(frame, (int(point[0] + x), int(point[1] + y)), 1, (255, 255, 255), 3)

    # Display the resulting frame
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

    cont += 1
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 15ms/step
